# Modeling3: 중복 feature 제거 X

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# 확인할 수 있는 열 갯수 늘리기
pd.set_option('display.max_columns', 35)

In [2]:
# 데이터 불러오기
data_origin = pd.read_csv("https://raw.githubusercontent.com/agtechresearch/LectureAlgorithm/main/csv/married_full.csv")
data_origin

,gender,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,eval_by_partner_attractive,eval_by_partner_sincere,eval_by_partner_intelligence,eval_by_partner_funny,eval_by_partner_ambitous,eval_by_partner_shared_interests,my_pref_attractive,my_pref_sincere,my_pref_intellicence,my_pref_funny,my_pref_ambtition,my_pref_shared_interests,my_eval_attractive,my_eval_sincere,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked,married
0,female,21.0,27.0,4.0,35.0,20.0,20.0,20.0,0.0,5.0,6.0,8.0,8.0,8.0,8.0,6.0,15.0,20.0,20.0,15.0,15.0,15.0,6.0,9.0,7.0,7.0,6.0,5.0,0.14,3.0,7.0,6.0,0
1,female,21.0,22.0,4.0,60.0,0.0,0.0,40.0,0.0,0.0,7.0,8.0,10.0,7.0,7.0,5.0,15.0,20.0,20.0,15.0,15.0,15.0,7.0,8.0,7.0,8.0,5.0,6.0,0.54,3.0,7.0,5.0,0
2,female,21.0,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.0,10.0,10.0,10.0,10.0,10.0,10.0,15.0,20.0,20.0,15.0,15.0,15.0,5.0,8.0,9.0,8.0,5.0,7.0,0.16,3.0,7.0,5.0,1
3,female,21.0,23.0,4.0,30.0,5.0,15.0,40.0,5.0,5.0,7.0,8.0,9.0,8.0,9.0,8.0,15.0,20.0,20.0,15.0,15.0,15.0,7.0,6.0,8.0,7.0,6.0,8.0,0.61,3.0,7.0,6.0,1
4,female,21.0,24.0,4.0,30.0,10.0,20.0,10.0,10.0,20.0,8.0,7.0,9.0,6.0,9.0,7.0,15.0,20.0,20.0,15.0,15.0,15.0,5.0,6.0,7.0,7.0,6.0,6.0,0.21,3.0,6.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,male,25.0,26.0,1.0,10.0,10.0,30.0,20.0,10.0,15.0,10.0,5.0,3.0,2.0,6.0,5.0,70.0,0.0,15.0,15.0,0.0,0.0,3.0,5.0,5.0,5.0,NaN,NaN,0.64,10.0,2.0,5.0,0
8358,male,25.0,24.0,1.0,50.0,20.0,10.0,5.0,10.0,5.0,6.0,3.0,7.0,3.0,7.0,2.0,70.0,0.0,15.0,15.0,0.0,0.0,4.0,6.0,8.0,4.0,4.0,NaN,0.71,10.0,4.0,4.0,0
8359,male,25.0,29.0,1.0,40.0,10.0,30.0,10.0,10.0,NaN,2.0,1.0,2.0,2.0,2.0,1.0,70.0,0.0,15.0,15.0,0.0,0.0,4.0,7.0,8.0,8.0,8.0,NaN,-0.46,10.0,6.0,5.0,0
8360,male,25.0,22.0,1.0,10.0,25.0,25.0,10.0,10.0,20.0,5.0,7.0,5.0,5.0,3.0,6.0,70.0,0.0,15.0,15.0,0.0,0.0,4.0,6.0,5.0,4.0,NaN,5.0,0.62,10.0,5.0,5.0,0


In [3]:
# 전처리를 위한 원본 데이터 복사
data = data_origin.copy()

# Preprocessing

> 전처리 process 정리
> 1. 분포가 겹치는 partner 데이터 모두 지우기
> 2. 범주형 변수 OneHotEncoding
> 3. 결측치 채우기(임의로 중앙값)
> 4. ADASYN을 통한 Class 불균형 해결
> 5. 수치형 변수 표준화(Standardization)

> 결측치 채우기(임시로 중앙값)

In [6]:
# 결측값 확인
data.isnull().sum()

gender                                0
age                                  72
age_partner                         102
importance_same_religion             62
pref_of_partner_attractive           83
pref_of_partner_sincere              83
pref_of_partner_intelligence         83
pref_of_partner_funny                91
pref_of_partner_ambitious            98
pref_of_partner_shared_interests    120
eval_by_partner_attractive          204
eval_by_partner_sincere             269
eval_by_partner_intelligence        291
eval_by_partner_funny               339
eval_by_partner_ambitous            681
eval_by_partner_shared_interests    989
my_pref_attractive                   62
my_pref_sincere                      62
my_pref_intellicence                 62
my_pref_funny                        72
my_pref_ambtition                    82
my_pref_shared_interests            104
my_eval_attractive                  198
my_eval_sincere                     271
my_eval_intelligence                286


In [7]:
# 결측치 중앙값으로 대체
median_value = data.iloc[:, 1:].median()
data.fillna(median_value, inplace=True)

In [8]:
data.isnull().sum()

gender                              0
age                                 0
age_partner                         0
importance_same_religion            0
pref_of_partner_attractive          0
pref_of_partner_sincere             0
pref_of_partner_intelligence        0
pref_of_partner_funny               0
pref_of_partner_ambitious           0
pref_of_partner_shared_interests    0
eval_by_partner_attractive          0
eval_by_partner_sincere             0
eval_by_partner_intelligence        0
eval_by_partner_funny               0
eval_by_partner_ambitous            0
eval_by_partner_shared_interests    0
my_pref_attractive                  0
my_pref_sincere                     0
my_pref_intellicence                0
my_pref_funny                       0
my_pref_ambtition                   0
my_pref_shared_interests            0
my_eval_attractive                  0
my_eval_sincere                     0
my_eval_intelligence                0
my_eval_funny                       0
my_eval_ambi

> 기본적인 데이터셋 준비 완료

In [9]:
data

,gender,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,eval_by_partner_attractive,eval_by_partner_sincere,eval_by_partner_intelligence,eval_by_partner_funny,eval_by_partner_ambitous,eval_by_partner_shared_interests,my_pref_attractive,my_pref_sincere,my_pref_intellicence,my_pref_funny,my_pref_ambtition,my_pref_shared_interests,my_eval_attractive,my_eval_sincere,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked,married
0,female,21.0,27.0,4.0,35.0,20.0,20.0,20.0,0.0,5.00,6.0,8.0,8.0,8.0,8.0,6.0,15.0,20.0,20.0,15.0,15.0,15.0,6.0,9.0,7.0,7.0,6.0,5.0,0.14,3.0,7.0,6.0,0
1,female,21.0,22.0,4.0,60.0,0.0,0.0,40.0,0.0,0.00,7.0,8.0,10.0,7.0,7.0,5.0,15.0,20.0,20.0,15.0,15.0,15.0,7.0,8.0,7.0,8.0,5.0,6.0,0.54,3.0,7.0,5.0,0
2,female,21.0,22.0,4.0,19.0,18.0,19.0,18.0,14.0,12.00,10.0,10.0,10.0,10.0,10.0,10.0,15.0,20.0,20.0,15.0,15.0,15.0,5.0,8.0,9.0,8.0,5.0,7.0,0.16,3.0,7.0,5.0,1
3,female,21.0,23.0,4.0,30.0,5.0,15.0,40.0,5.0,5.00,7.0,8.0,9.0,8.0,9.0,8.0,15.0,20.0,20.0,15.0,15.0,15.0,7.0,6.0,8.0,7.0,6.0,8.0,0.61,3.0,7.0,6.0,1
4,female,21.0,24.0,4.0,30.0,10.0,20.0,10.0,10.0,20.00,8.0,7.0,9.0,6.0,9.0,7.0,15.0,20.0,20.0,15.0,15.0,15.0,5.0,6.0,7.0,7.0,6.0,6.0,0.21,3.0,6.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,male,25.0,26.0,1.0,10.0,10.0,30.0,20.0,10.0,15.00,10.0,5.0,3.0,2.0,6.0,5.0,70.0,0.0,15.0,15.0,0.0,0.0,3.0,5.0,5.0,5.0,7.0,6.0,0.64,10.0,2.0,5.0,0
8358,male,25.0,24.0,1.0,50.0,20.0,10.0,5.0,10.0,5.00,6.0,3.0,7.0,3.0,7.0,2.0,70.0,0.0,15.0,15.0,0.0,0.0,4.0,6.0,8.0,4.0,4.0,6.0,0.71,10.0,4.0,4.0,0
8359,male,25.0,29.0,1.0,40.0,10.0,30.0,10.0,10.0,10.64,2.0,1.0,2.0,2.0,2.0,1.0,70.0,0.0,15.0,15.0,0.0,0.0,4.0,7.0,8.0,8.0,8.0,6.0,-0.46,10.0,6.0,5.0,0
8360,male,25.0,22.0,1.0,10.0,25.0,25.0,10.0,10.0,20.00,5.0,7.0,5.0,5.0,3.0,6.0,70.0,0.0,15.0,15.0,0.0,0.0,4.0,6.0,5.0,4.0,7.0,5.0,0.62,10.0,5.0,5.0,0


In [10]:
# 메모리를 효율적으로 사용하기 위한 downcast 함수 정의
def downcast(df, verbose=True):     # verbose 옵션 추가: (True)인 경우 몇 퍼센트 압축됐는지 출력
    start_mem = df.memory_usage().sum() / 1024**2   # 초기 메모리 사용량
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2

    if verbose:
        print(f'{(100*(start_mem - end_mem) / start_mem):.1f}% 압축됨')

    return df

In [11]:
downcast(data)

54.2% 압축됨


,gender,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,eval_by_partner_attractive,eval_by_partner_sincere,eval_by_partner_intelligence,eval_by_partner_funny,eval_by_partner_ambitous,eval_by_partner_shared_interests,my_pref_attractive,my_pref_sincere,my_pref_intellicence,my_pref_funny,my_pref_ambtition,my_pref_shared_interests,my_eval_attractive,my_eval_sincere,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked,married
0,female,21,27,4,35.0,20.0,20.0,20.0,0.0,5.00,6.0,8.0,8.0,8.0,8.0,6.0,15.0,20.0,20.0,15.0,15.0,15.0,6.0,9.0,7.0,7.0,6.0,5.0,0.14,3,7.0,6.0,0
1,female,21,22,4,60.0,0.0,0.0,40.0,0.0,0.00,7.0,8.0,10.0,7.0,7.0,5.0,15.0,20.0,20.0,15.0,15.0,15.0,7.0,8.0,7.0,8.0,5.0,6.0,0.54,3,7.0,5.0,0
2,female,21,22,4,19.0,18.0,19.0,18.0,14.0,12.00,10.0,10.0,10.0,10.0,10.0,10.0,15.0,20.0,20.0,15.0,15.0,15.0,5.0,8.0,9.0,8.0,5.0,7.0,0.16,3,7.0,5.0,1
3,female,21,23,4,30.0,5.0,15.0,40.0,5.0,5.00,7.0,8.0,9.0,8.0,9.0,8.0,15.0,20.0,20.0,15.0,15.0,15.0,7.0,6.0,8.0,7.0,6.0,8.0,0.61,3,7.0,6.0,1
4,female,21,24,4,30.0,10.0,20.0,10.0,10.0,20.00,8.0,7.0,9.0,6.0,9.0,7.0,15.0,20.0,20.0,15.0,15.0,15.0,5.0,6.0,7.0,7.0,6.0,6.0,0.21,3,6.0,6.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,male,25,26,1,10.0,10.0,30.0,20.0,10.0,15.00,10.0,5.0,3.0,2.0,6.0,5.0,70.0,0.0,15.0,15.0,0.0,0.0,3.0,5.0,5.0,5.0,7.0,6.0,0.64,10,2.0,5.0,0
8358,male,25,24,1,50.0,20.0,10.0,5.0,10.0,5.00,6.0,3.0,7.0,3.0,7.0,2.0,70.0,0.0,15.0,15.0,0.0,0.0,4.0,6.0,8.0,4.0,4.0,6.0,0.71,10,4.0,4.0,0
8359,male,25,29,1,40.0,10.0,30.0,10.0,10.0,10.64,2.0,1.0,2.0,2.0,2.0,1.0,70.0,0.0,15.0,15.0,0.0,0.0,4.0,7.0,8.0,8.0,8.0,6.0,-0.46,10,6.0,5.0,0
8360,male,25,22,1,10.0,25.0,25.0,10.0,10.0,20.00,5.0,7.0,5.0,5.0,3.0,6.0,70.0,0.0,15.0,15.0,0.0,0.0,4.0,6.0,5.0,4.0,7.0,5.0,0.62,10,5.0,5.0,0


# Scaling

In [12]:
# train feature와 target 데이터 분리
X = data.drop(['married'], axis=1)
y = data['married']

In [14]:
# 수치형 변수들 Standardization
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X.iloc[:, 1:] = scaler.fit_transform(X.iloc[:, 1:])
X

,gender,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,eval_by_partner_attractive,eval_by_partner_sincere,eval_by_partner_intelligence,eval_by_partner_funny,eval_by_partner_ambitous,eval_by_partner_shared_interests,my_pref_attractive,my_pref_sincere,my_pref_intellicence,my_pref_funny,my_pref_ambtition,my_pref_shared_interests,my_eval_attractive,my_eval_sincere,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked
0,female,-1.505599,0.180042,0.126303,1.002150,0.371459,-0.038888,0.419362,-1.791353,-1.081836,-0.095444,0.486658,0.423968,0.822577,0.702992,0.227499,-0.597651,0.370057,-0.038927,-0.406326,0.730873,0.501125,-0.098079,1.068271,-0.232671,0.298978,-0.462444,-0.266296,-0.187910,-1.462598,0.477257,0.381072
1,female,-1.505599,-1.230934,0.126303,3.000178,-2.482603,-3.002747,3.725149,-1.791353,-1.873651,0.423510,0.486658,1.733857,0.300934,0.121460,-0.266938,-0.597651,0.370057,-0.038927,-0.406326,0.730873,0.501125,0.421191,0.484403,-0.232671,0.820639,-1.044028,0.227389,1.140970,-1.462598,0.477257,-0.096632
2,female,-1.505599,-1.230934,0.126303,-0.276588,0.086052,-0.187081,0.088784,0.559954,0.026705,1.980372,1.654190,1.733857,1.865863,1.866057,2.205245,-0.597651,0.370057,-0.038927,-0.406326,0.730873,0.501125,-0.617348,0.484403,1.077715,0.820639,-1.044028,0.721073,-0.121466,-1.462598,0.477257,-0.096632
3,female,-1.505599,-0.948739,0.126303,0.602544,-1.769088,-0.779853,3.725149,-0.951601,-1.081836,0.423510,0.486658,1.078912,0.822577,1.284525,1.216372,-0.597651,0.370057,-0.038927,-0.406326,0.730873,0.501125,0.421191,-0.683332,0.422522,0.298978,-0.462444,1.214758,1.373524,-1.462598,0.477257,0.381072
4,female,-1.505599,-0.666544,0.126303,0.602544,-1.055572,-0.038888,-1.233531,-0.111848,1.293610,0.942464,-0.097108,1.078912,-0.220710,1.284525,0.721935,-0.597651,0.370057,-0.038927,-0.406326,0.730873,0.501125,-0.617348,-0.683332,-0.232671,0.298978,-0.462444,0.227389,0.044644,-1.462598,-0.073977,0.381072
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,male,-0.379975,-0.102153,-0.947012,-0.995878,-1.055572,1.443041,0.419362,-0.111848,0.501795,1.980372,-1.264641,-2.850757,-2.307281,-0.460073,-0.266938,3.787828,-2.478944,-0.778872,-0.406326,-1.788581,-1.870912,-1.655887,-1.267200,-1.543057,-0.744342,0.119141,0.227389,1.473190,2.582369,-2.278913,-0.096632
8358,male,-0.379975,-0.666544,-0.947012,2.200966,0.371459,-1.520817,-2.059978,-0.111848,-1.081836,-0.095444,-2.432173,-0.230977,-1.785639,0.121460,-1.750248,3.787828,-2.478944,-0.778872,-0.406326,-1.788581,-1.870912,-1.136617,-0.683332,0.422522,-1.266002,-1.625612,0.227389,1.705743,2.582369,-1.176445,-0.574336
8359,male,-0.379975,0.744432,-0.947012,1.401755,-1.055572,1.443041,-1.233531,-0.111848,-0.188668,-2.171259,-3.599705,-3.505702,-2.307281,-2.786203,-2.244684,3.787828,-2.478944,-0.778872,-0.406326,-1.788581,-1.870912,-1.136617,-0.099464,0.422522,0.820639,0.700725,0.227389,-2.181229,2.582369,-0.073977,-0.096632
8360,male,-0.379975,-1.230934,-0.947012,-0.995878,1.084974,0.702076,-1.233531,-0.111848,1.293610,-0.614397,-0.097108,-1.540867,-0.742353,-2.204670,0.227499,3.787828,-2.478944,-0.778872,-0.406326,-1.788581,-1.870912,-1.136617,-0.683332,-1.543057,-1.266002,0.119141,-0.266296,1.406746,2.582369,-0.625211,-0.096632


In [15]:
# gender One-hot encoding
X['gender_male'] = pd.get_dummies(X['gender'], drop_first=True)
X.drop('gender', axis=1, inplace=True)

In [16]:
X

,age,age_partner,importance_same_religion,pref_of_partner_attractive,pref_of_partner_sincere,pref_of_partner_intelligence,pref_of_partner_funny,pref_of_partner_ambitious,pref_of_partner_shared_interests,eval_by_partner_attractive,eval_by_partner_sincere,eval_by_partner_intelligence,eval_by_partner_funny,eval_by_partner_ambitous,eval_by_partner_shared_interests,my_pref_attractive,my_pref_sincere,my_pref_intellicence,my_pref_funny,my_pref_ambtition,my_pref_shared_interests,my_eval_attractive,my_eval_sincere,my_eval_intelligence,my_eval_funny,my_eval_ambition,my_eval_shared_interests,interests_correlate,expected_happy_with_couple_match,how_much_i_liked,guess_prob_liked,gender_male
0,-1.505599,0.180042,0.126303,1.002150,0.371459,-0.038888,0.419362,-1.791353,-1.081836,-0.095444,0.486658,0.423968,0.822577,0.702992,0.227499,-0.597651,0.370057,-0.038927,-0.406326,0.730873,0.501125,-0.098079,1.068271,-0.232671,0.298978,-0.462444,-0.266296,-0.187910,-1.462598,0.477257,0.381072,False
1,-1.505599,-1.230934,0.126303,3.000178,-2.482603,-3.002747,3.725149,-1.791353,-1.873651,0.423510,0.486658,1.733857,0.300934,0.121460,-0.266938,-0.597651,0.370057,-0.038927,-0.406326,0.730873,0.501125,0.421191,0.484403,-0.232671,0.820639,-1.044028,0.227389,1.140970,-1.462598,0.477257,-0.096632,False
2,-1.505599,-1.230934,0.126303,-0.276588,0.086052,-0.187081,0.088784,0.559954,0.026705,1.980372,1.654190,1.733857,1.865863,1.866057,2.205245,-0.597651,0.370057,-0.038927,-0.406326,0.730873,0.501125,-0.617348,0.484403,1.077715,0.820639,-1.044028,0.721073,-0.121466,-1.462598,0.477257,-0.096632,False
3,-1.505599,-0.948739,0.126303,0.602544,-1.769088,-0.779853,3.725149,-0.951601,-1.081836,0.423510,0.486658,1.078912,0.822577,1.284525,1.216372,-0.597651,0.370057,-0.038927,-0.406326,0.730873,0.501125,0.421191,-0.683332,0.422522,0.298978,-0.462444,1.214758,1.373524,-1.462598,0.477257,0.381072,False
4,-1.505599,-0.666544,0.126303,0.602544,-1.055572,-0.038888,-1.233531,-0.111848,1.293610,0.942464,-0.097108,1.078912,-0.220710,1.284525,0.721935,-0.597651,0.370057,-0.038927,-0.406326,0.730873,0.501125,-0.617348,-0.683332,-0.232671,0.298978,-0.462444,0.227389,0.044644,-1.462598,-0.073977,0.381072,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8357,-0.379975,-0.102153,-0.947012,-0.995878,-1.055572,1.443041,0.419362,-0.111848,0.501795,1.980372,-1.264641,-2.850757,-2.307281,-0.460073,-0.266938,3.787828,-2.478944,-0.778872,-0.406326,-1.788581,-1.870912,-1.655887,-1.267200,-1.543057,-0.744342,0.119141,0.227389,1.473190,2.582369,-2.278913,-0.096632,True
8358,-0.379975,-0.666544,-0.947012,2.200966,0.371459,-1.520817,-2.059978,-0.111848,-1.081836,-0.095444,-2.432173,-0.230977,-1.785639,0.121460,-1.750248,3.787828,-2.478944,-0.778872,-0.406326,-1.788581,-1.870912,-1.136617,-0.683332,0.422522,-1.266002,-1.625612,0.227389,1.705743,2.582369,-1.176445,-0.574336,True
8359,-0.379975,0.744432,-0.947012,1.401755,-1.055572,1.443041,-1.233531,-0.111848,-0.188668,-2.171259,-3.599705,-3.505702,-2.307281,-2.786203,-2.244684,3.787828,-2.478944,-0.778872,-0.406326,-1.788581,-1.870912,-1.136617,-0.099464,0.422522,0.820639,0.700725,0.227389,-2.181229,2.582369,-0.073977,-0.096632,True
8360,-0.379975,-1.230934,-0.947012,-0.995878,1.084974,0.702076,-1.233531,-0.111848,1.293610,-0.614397,-0.097108,-1.540867,-0.742353,-2.204670,0.227499,3.787828,-2.478944,-0.778872,-0.406326,-1.788581,-1.870912,-1.136617,-0.683332,-1.543057,-1.266002,0.119141,-0.266296,1.406746,2.582369,-0.625211,-0.096632,True


> 데이터 불균형 문제가 심각하다. 별도의 처리가 필요할 것 같다.

In [17]:
# SMOTE 샘플링
#from imblearn.over_sampling import SMOTE

#smote = SMOTE(random_state=42)
#X_resampled, y_resampled = smote.fit_resample(X, y)


# ADASYN 샘플링
#from imblearn.over_sampling import ADASYN

#adasyn = ADASYN(random_state=42)
#X_resampled, y_resampled = adasyn.fit_resample(X, y)


#SMOTE-Tomek 샘플링
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import TomekLinks

smoteto = SMOTETomek(tomek=TomekLinks(sampling_strategy='majority'))
X_resampled, y_resampled = smoteto.fit_resample(X, y)

In [18]:
X_resampled.shape, y_resampled.shape

((13693, 32), (13693,))

# Model Training

In [53]:
# 원본 데이터 train, test 데이터 분리
from sklearn.model_selection import train_test_split
X_train_origin, X_test_origin, y_train_origin, y_test_origin = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_origin.shape, X_test_origin.shape, y_train_origin.shape, y_test_origin.shape

((6689, 32), (1673, 32), (6689,), (1673,))

In [54]:
# 오버샘플링한 train, test 데이터 분리
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((10955, 32), (2739, 32), (10955,), (2739,))

In [55]:
# lightgbm 모델 학습
from lightgbm import LGBMClassifier

params = {
    'n_estimators': 300,        #* 반복 수행하려는 트리의 개수 (너무 크면 오버피팅 발생)
    'max_depth': 8,             #* 트리의 최대 깊이 (너무 크면 오버피팅 발생)   # 보통 3~12(가장 민감하므로 먼저 튜닝해야할 값) # lightgbm처럼 leaf-wise로 학습하는 경우 끝까지(-1)도 괜찮다고 한다.
    # 'boosting_type' = 'gbdt'  # 부스팅 타입 (gbdt: Gradient Boosting Decision Tree, rf: RandomForest)
    'num_leaves': 16,           #* 하나의 트리가 가질 수 있는 최대 리프 수 (2번째로 민감. max_depth와 함께 튜닝)
    'learning_rate': 0.1,       # 부스팅 스탭 반복할 때 학습률(0~1) #튜닝시 0.1~0.3, 최종 모형은 0.05 이하
}

lgbm = LGBMClassifier(**params, objective='binary', metric='accuracy', random_state=42, verbose=0)
lgbm.fit(X_train, y_train)

LGBMClassifier(max_depth=8, metric='accuracy', n_estimators=300, num_leaves=16,
               objective='binary', random_state=42, verbose=0)

In [56]:
# 성능 평가
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

y_train_pred = lgbm.predict(X_train)
print("<<Train Set Performance>>")
print("Accuracy: ", accuracy_score(y_train, y_train_pred))
print("F1 Score: ", f1_score(y_train, y_train_pred))
print(confusion_matrix(y_train, y_train_pred))
print('--------'*10)

y_test_pred = lgbm.predict(X_test_origin)
print("<<Test Set Performance>>")
print("Accuracy: ", accuracy_score(y_test_origin, y_test_pred))
print("F1 Score: ", f1_score(y_test_origin, y_test_pred))
print(confusion_matrix(y_test_origin, y_test_pred))

<<Train Set Performance>>
Accuracy:  0.9810132359653126
F1 Score:  0.9808753218094888
[[5413   39]
 [ 169 5334]]
--------------------------------------------------------------------------------
<<Test Set Performance>>
Accuracy:  0.9503885236102809
F1 Score:  0.857142857142857
[[1341   26]
 [  57  249]]


In [57]:
# 각종 성능 지표 확인
from sklearn.metrics import classification_report

print(classification_report(y_test_origin, y_test_pred))

              precision    recall  f1-score   support

           0       0.96      0.98      0.97      1367
           1       0.91      0.81      0.86       306

    accuracy                           0.95      1673
   macro avg       0.93      0.90      0.91      1673
weighted avg       0.95      0.95      0.95      1673

